In [3]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind
from scipy.stats import ttest_rel
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.stats.multitest as multi
import array
import math
%matplotlib inline

In [4]:
healthy_df = pd.read_csv("lusc-rsem-fpkm-tcga_paired.txt", sep = '\t')

In [96]:
healthy_df.head()

,Hugo_Symbol,Entrez_Gene_Id,TCGA-43-7657,TCGA-58-8386,TCGA-22-5478,TCGA-22-5472,TCGA-43-5670,TCGA-60-2709,TCGA-22-5489,TCGA-77-8007,...,TCGA-85-7710,TCGA-56-7580,TCGA-43-6647,TCGA-90-6837,TCGA-56-8083,TCGA-51-4079,TCGA-56-7222,TCGA-22-4593,TCGA-51-4081,TCGA-56-8309
0,HIST3H2A,92815,62.12,130.60,33.06,35.50,73.03,60.39,92.05,66.65,...,90.77,59.55,40.07,22.92,29.91,82.29,4.70,37.32,43.63,77.25
1,LIN7B,64130,185.11,283.05,119.26,169.07,165.57,161.02,131.51,198.47,...,185.11,119.26,102.97,123.50,264.03,194.36,166.73,105.15,185.11,356.05
2,LXN,56925,909.17,819.30,412.00,743.43,1340.84,607.87,1709.26,1709.26,...,813.63,2400.97,543.96,2193.99,540.19,521.76,253.23,764.36,518.15,878.17
3,CNKSR2,22866,41.81,18.29,40.93,67.12,54.72,29.27,20.26,23.76,...,34.51,70.01,57.49,57.89,67.12,34.51,22.10,31.90,28.24,49.91
4,SCML1,6322,133.36,214.27,108.14,109.66,190.34,211.31,96.01,208.38,...,251.48,209.84,120.10,109.66,155.50,162.14,277.20,86.43,164.42,155.50


In [103]:
np.mean(array)

19364.11150642484

In [6]:
cancer_df = pd.read_csv("lusc-rsem-fpkm-tcga-t_paired.txt", sep = '\t')

In [7]:
cancer_df.head()

,Hugo_Symbol,Entrez_Gene_Id,TCGA-43-7657,TCGA-58-8386,TCGA-22-5478,TCGA-22-5472,TCGA-43-5670,TCGA-60-2709,TCGA-22-5489,TCGA-77-8007,...,TCGA-85-7710,TCGA-56-7580,TCGA-43-6647,TCGA-90-6837,TCGA-56-8083,TCGA-51-4079,TCGA-56-7222,TCGA-22-4593,TCGA-51-4081,TCGA-56-8309
0,HIST3H2A,92815,336.79,500.46,703.28,287.01,486.75,70.51,145.02,14.03,...,3.06,420.68,109.66,106.63,1233.75,172.65,303.44,228.13,251.48,23.59
1,LIN7B,64130,105.15,212.78,102.25,212.78,172.65,244.57,105.89,152.28,...,135.24,135.24,151.22,395.18,295.11,120.94,114.36,96.68,277.20,153.34
2,LXN,56925,848.22,236.21,271.48,759.08,61.25,620.67,329.84,599.49,...,688.78,204.07,438.59,503.95,3039.30,607.87,106.63,507.46,255.00,346.29
3,CNKSR2,22866,32.59,8.51,45.85,6.16,49.21,11.91,12.27,15.00,...,1.38,6.62,6.11,1.66,33.54,3.11,0.82,3.32,27.25,6.16
4,SCML1,6322,84.63,74.58,67.12,57.89,102.97,132.44,66.65,57.08,...,165.57,119.26,87.65,53.57,232.94,67.12,64.80,90.14,198.47,154.42


In [8]:
healthy_df.shape, cancer_df.shape

((19648, 52), (19648, 52))

In [9]:
ZERO_THRESHOLD = 30

In [10]:
def contain_zeros(h_series, c_series):
    """
    contain_zeros: a function that counts the zeros in both cancer and healthy series and
    returns True if either exceeds ZERO_THRESHOLD, False otherwise
    """
    h_zeros = 0
    c_zeros = 0
    h_value_counts = h_series.value_counts()
    c_value_counts = c_series.value_counts()
    if 0 in h_value_counts.index:
        h_zeros = h_value_counts[0]
    else:
        h_zeros = 0
    if 0 in c_value_counts.index:
        c_zeros = c_value_counts[0]
    else:
        c_zeros = 0
        
    return (h_zeros > ZERO_THRESHOLD or c_zeros > ZERO_THRESHOLD)

In [11]:
   def plot_distribution(inp):
    plt.figure()
    ax = sns.distplot(inp)
    plt.axvline(np.mean(inp), color="k", linestyle="dashed", linewidth=5)
    _, max_ = plt.ylim()
    plt.text(
        inp.mean() + inp.mean() / 10,
        max_ - max_ / 10,
        "Mean: {:.2f}".format(inp.mean()),
    )
    return plt.figure

In [12]:
#samples are independent

In [118]:
p_value_dict = {}
mean_healthy_array=[]
mean_cancer_array=[]

In [119]:
for i in range(healthy_df.shape[0]):
    gene_type = healthy_df.iloc[i, 0]
    healthy_values = healthy_df.iloc[i, 2:].astype(np.float64)
    cancer_values = cancer_df.iloc[i, 2:].astype(np.float64)
    zeros_status = contain_zeros(healthy_values, cancer_values)
    if not zeros_status: #if the number of zeroes in both series dosent exceed 30
        p_value= ttest_ind(healthy_values, cancer_values).pvalue
        mean_healthy = np.mean(healthy_values)
        mean_cancer = np.mean(cancer_values)
        if not math.isnan(p_value): #to ensure that the p_value is not null
            p_value_dict[gene_type] = p_value
            mean_healthy_array.append(mean_healthy)
            mean_cancer_array.append(mean_cancer)

In [120]:
mean_healthy_array

[53.66600000000001,
 171.9386,
 866.3358000000002,
 41.534000000000006,
 167.0196,
 27.265,
 2125.0454000000004,
 994.0726,
 147.45960000000002,
 1419.0256,
 205.09179999999998,
 377.90340000000003,
 49.46540000000001,
 696.3764000000001,
 33.0716,
 939.2252000000002,
 111.8178,
 310.2102000000001,
 250.72120000000004,
 315.9758000000001,
 896.7696000000001,
 1580.7002000000007,
 482.0768,
 361.08760000000007,
 1909.7876,
 2446.005,
 539.1066,
 46869.947000000015,
 33.7102,
 115.33660000000002,
 8031.9317999999985,
 19.717200000000002,
 32299.173200000005,
 605.6506000000002,
 1571.6229999999998,
 14.3608,
 149.90640000000005,
 142.20480000000003,
 338.809,
 868.9908000000001,
 559.3780000000002,
 1438.5511999999997,
 549.5744000000001,
 981.5146000000003,
 599.9668,
 851.8948000000001,
 96.65220000000004,
 435.5962000000001,
 2214.8433999999997,
 95.26440000000002,
 1926.1801999999998,
 532.3704,
 124.62360000000001,
 7988.977799999999,
 88.32519999999998,
 873.1902,
 1831.67500000000

In [121]:
np.mean(mean_healthy_array)

1392.3288226250495

In [122]:
mean_cancer_array

[286.45339999999993,
 187.44820000000004,
 490.3438,
 14.0076,
 138.465,
 33.58020000000001,
 1525.3039999999999,
 12934.283000000001,
 93.98960000000002,
 654.4714,
 152.9592,
 683.7959999999999,
 95.70300000000003,
 352.80119999999994,
 12.449000000000002,
 960.4753999999999,
 175.85760000000005,
 196.46120000000002,
 1130.7086,
 327.3488,
 1130.7459999999999,
 1305.7454,
 516.0686000000002,
 101.84180000000003,
 3725.5432,
 4529.500600000001,
 426.32920000000007,
 45138.874800000005,
 55.93919999999999,
 125.04700000000001,
 4758.2464,
 304.6654000000001,
 2821.4098,
 1011.9320000000004,
 1885.2767999999996,
 25.314800000000005,
 196.81000000000006,
 185.5382,
 199.82639999999995,
 4040.088999999999,
 511.4868000000001,
 1162.6432,
 959.2648000000004,
 302.9800000000001,
 592.1432000000001,
 458.8351999999998,
 13.337200000000001,
 288.74319999999994,
 1916.9882000000005,
 73.9948,
 2549.640000000001,
 698.9368000000002,
 1044.7144,
 10535.936399999999,
 59.85759999999999,
 1563.302

In [123]:
np.mean(mean_cancer_array)

1199.021514178893

In [15]:
plt.figure()
ax1 = sns.distplot(healthy_values)
ax2 = sns.distplot(cancer_values)
plt.axvline(np.mean(healtfor gene, pearson_coef in sorted_pearson_coef_dict.items():hy_values), color='b', linestyle='dashed', linewidth=5)
plt.axvline(np.mean(cancer_values), color='orange', linestyle='dashed', linewidth=5)

SyntaxError: invalid syntax (<ipython-input-15-3ff06aa1c284>, line 4)

In [16]:
p_value_dict

{'HIST3H2A': 3.607140106656369e-09,
 'LIN7B': 0.31382948251004256,
 'LXN': 8.164043644485835e-05,
 'CNKSR2': 6.3746517799747675e-15,
 'SCML1': 0.04726589843891226,
 'AC024592.12': 0.06031615049208138,
 'GSDMD': 5.344288657720391e-06,
 'AKR1C1': 7.857877107856742e-06,
 'C3orf62': 1.5007152753534581e-09,
 'CRISPLD2': 9.742753347580845e-05,
 'DOCK5': 0.046715788790729566,
 'SLC33A1': 6.382057732213531e-11,
 'GLI1': 0.023980398417391267,
 'STK17B': 5.024134511809196e-13,
 'RP5-966M1.6': 1.7233551280507414e-09,
 'VPS52': 0.6096108831192029,
 'HAGHL': 0.009115074132559457,
 'CDC37L1': 3.156672288692492e-16,
 'KRT10': 0.0003716169034954075,
 'AEN': 0.6618238617557775,
 'CYHR1': 0.00027796006033760834,
 'VWA1': 0.35764236729681453,
 'EXOSC9': 0.21467447604795004,
 'TNFAIP8L2': 5.504189276736617e-23,
 'PFDN2': 2.225416254484468e-16,
 'TCP1': 9.875245155160994e-12,
 'ALG9': 3.2608707729456655e-08,
 'RPS15A': 0.6025969151731221,
 'PMEL': 0.013851285748684753,
 'SFMBT1': 0.24280967106329254,
 'CYB

In [17]:
len(p_value_dict)

17653

In [26]:
rejected_genes = {}
rejected_gene_names=[]
for gene, p_value in p_value_dict.items():
    if (p_value < 0.05):
        rejected_genes[gene] = 'Reject H0'
        rejected_gene_names.append(gene)
        #print(gene,'Reject H0')
    #else:
        #print(gene,' Fail To Reject Ho')

In [27]:
rejected_genes

{'HIST3H2A': 'Reject H0',
 'LXN': 'Reject H0',
 'CNKSR2': 'Reject H0',
 'SCML1': 'Reject H0',
 'GSDMD': 'Reject H0',
 'AKR1C1': 'Reject H0',
 'C3orf62': 'Reject H0',
 'CRISPLD2': 'Reject H0',
 'DOCK5': 'Reject H0',
 'SLC33A1': 'Reject H0',
 'GLI1': 'Reject H0',
 'STK17B': 'Reject H0',
 'RP5-966M1.6': 'Reject H0',
 'HAGHL': 'Reject H0',
 'CDC37L1': 'Reject H0',
 'KRT10': 'Reject H0',
 'CYHR1': 'Reject H0',
 'TNFAIP8L2': 'Reject H0',
 'PFDN2': 'Reject H0',
 'TCP1': 'Reject H0',
 'ALG9': 'Reject H0',
 'PMEL': 'Reject H0',
 'CYBA': 'Reject H0',
 'GINS1': 'Reject H0',
 'GPX3': 'Reject H0',
 'IPO9': 'Reject H0',
 'SHKBP1': 'Reject H0',
 'ARHGAP40': 'Reject H0',
 'ENO3': 'Reject H0',
 'COX19': 'Reject H0',
 'LMF1': 'Reject H0',
 'GDE1': 'Reject H0',
 'LSM14B': 'Reject H0',
 'TLE2': 'Reject H0',
 'EIF4EBP3': 'Reject H0',
 'LRP2BP': 'Reject H0',
 'PELO': 'Reject H0',
 'CDIPT': 'Reject H0',
 'C19orf44': 'Reject H0',
 'PABPN1': 'Reject H0',
 'ZWINT': 'Reject H0',
 'DYNLL1': 'Reject H0',
 'NRXN3':

In [28]:
len(rejected_genes)

12799

In [31]:
rejected_gene_names

['HIST3H2A',
 'LXN',
 'CNKSR2',
 'SCML1',
 'GSDMD',
 'AKR1C1',
 'C3orf62',
 'CRISPLD2',
 'DOCK5',
 'SLC33A1',
 'GLI1',
 'STK17B',
 'RP5-966M1.6',
 'HAGHL',
 'CDC37L1',
 'KRT10',
 'CYHR1',
 'TNFAIP8L2',
 'PFDN2',
 'TCP1',
 'ALG9',
 'PMEL',
 'CYBA',
 'GINS1',
 'GPX3',
 'IPO9',
 'SHKBP1',
 'ARHGAP40',
 'ENO3',
 'COX19',
 'LMF1',
 'GDE1',
 'LSM14B',
 'TLE2',
 'EIF4EBP3',
 'LRP2BP',
 'PELO',
 'CDIPT',
 'C19orf44',
 'PABPN1',
 'ZWINT',
 'DYNLL1',
 'NRXN3',
 'FYTTD1',
 'CAPRIN1',
 'AOC3',
 'KIFC3',
 'DOCK11',
 'TMEM231',
 'DNAH17',
 'DCHS1',
 'ZFYVE19',
 'HS3ST4',
 'SLC5A8',
 'WDR53',
 'NLN',
 'TBC1D31',
 'ZNF597',
 'RNF114',
 'SFRP2',
 'PNMA2',
 'C1orf158',
 'HENMT1',
 'STX16',
 'BFAR',
 'SHROOM3',
 'NUP107',
 'HS6ST1',
 'BRWD3',
 'FGF11',
 'CCDC138',
 'FAM193B',
 'BATF',
 'AGFG2',
 'TVP23B',
 'BCL6',
 'DNAJC3',
 'C1orf233',
 'ADH6',
 'ITGB6',
 'CEACAM21',
 'ETV6',
 'C1RL',
 'CCDC181',
 'GPN3',
 'RAP1GAP',
 'MAGED2',
 'C1orf85',
 'JRKL',
 'BHLHE40',
 'NMUR1',
 'SPATA18',
 'SLC17A3',
 'NTPCR'

In [32]:
#FDR Correction Method

In [33]:
P_ind=[]
gene_names_ind=[]

In [34]:
for gene, p_value in p_value_dict.items():
    P_ind.append(p_value)
    gene_names_ind.append(gene)

In [35]:
len(P_ind)

17653

In [36]:
q_fdr_ind = multi.multipletests(P_ind, method = 'fdr_bh')[1]

In [37]:
q_fdr_ind

array([1.39918357e-08, 3.64475780e-01, 1.74015772e-04, ...,
       9.50928331e-14, 6.40570927e-12, 6.53576363e-07])

In [38]:
len(q_fdr_ind)

17653

In [39]:
gene_names_modified_ind = [gene_names_ind[x] for x in range(len(P_ind)) if q_fdr_ind[x]<=0.05]

In [40]:
#number of rejected genes after correction
len(gene_names_modified_ind)

12480

In [41]:
#number of rejected genes before correction
len(rejected_genes) 

12799

In [42]:
intersection=[]
intersection=np.intersect1d(rejected_gene_names,gene_names_modified_ind)

In [43]:
intersection

array(['A2M', 'A2ML1', 'A4GALT', ..., 'ZZEF1', 'ZZZ3', 'hsa-mir-1199'],
      dtype='<U17')

In [44]:
len(intersection)

12480

In [48]:
difference=[]
difference = np.setdiff1d(rejected_gene_names,gene_names_modified_ind)

In [49]:
difference

array(['AC005606.1', 'AC008132.13', 'AC011294.3', 'ACACA', 'ACAP1',
       'ACTL10', 'ADAM28', 'AGAP6', 'AGBL2', 'AGL', 'AICDA', 'AL020996.1',
       'AL354808.2', 'AL359878.1', 'AL591684.1', 'ALDH1L1', 'AMDHD1',
       'ANKMY2', 'ANKRD34B', 'ANKRD36', 'AOC1', 'APBA1', 'ARFGEF2',
       'ARL14EP', 'ARL3', 'ARMC12', 'ASB7', 'ATP12A', 'ATP1A3', 'ATP5A1',
       'B3GALT1', 'BAI1', 'BCAR3', 'BCLAF1', 'BRDT', 'C10orf82',
       'C12orf71', 'C20orf201', 'C2CD3', 'C3orf70', 'CA7', 'CALCR',
       'CATSPER3', 'CCL7', 'CCNB1IP1', 'CD1A', 'CD320', 'CD70', 'CD79B',
       'CDK3', 'CDR2L', 'CENPBD1', 'CERK', 'CHRM5', 'CHSY1', 'CHURC1',
       'CLDN15', 'CLEC4G', 'CLMN', 'CLPTM1', 'COBLL1', 'COL11A2',
       'COL19A1', 'COPB1', 'CSNK1D', 'CTCF', 'CWF19L2', 'CXCR4', 'CXXC1',
       'CYP1A1', 'CYP46A1', 'DBT', 'DCT', 'DCTN2', 'DDHD2', 'DEF6',
       'DHRS4-AS1', 'DIDO1', 'DLX5', 'DMAP1', 'DMD', 'DMKN', 'DOCK5',
       'DSG1', 'DUOX2', 'DUSP23', 'DYNLT3', 'DZIP1', 'ECD', 'EGFLAM',
       'EHBP1L1', 'E

In [50]:
len(difference)

319

In [51]:
#samples are paired

In [134]:
p_value_dict_paired = {}
mean_healthy_array_paired=[]
mean_cancer_array_paired=[]
mean_healthy_cancer_array=[]

In [138]:
for i in range(healthy_df.shape[0]):
    gene_type = healthy_df.iloc[i, 0]
    healthy_values = healthy_df.iloc[i, 2:].astype(np.float64)
    cancer_values = cancer_df.iloc[i, 2:].astype(np.float64)
    zeros_status = contain_zeros(healthy_values, cancer_values)
    if not zeros_status: #if the number of zeroes in both series dosent exceed 30
        p_value_paired= ttest_rel(healthy_values, cancer_values).pvalue
        mean_healthy_paired= np.mean(healthy_values)
        mean_cancer_paired= np.mean(cancer_values)
        mean_healthy_cancer_paired=mean_healthy_paired - mean_cancer_paired 
        if not math.isnan(p_value_paired): #to ensure that the p_value is not null
            p_value_dict_paired[gene_type] = p_value_paired
            mean_healthy_array_paired.append(mean_healthy_paired)
            mean_cancer_array_paired.append(mean_cancer_paired)
            mean_healthy_cancer_array.append(mean_healthy_cancer_paired)

In [139]:
 mean_healthy_cancer_array

[-232.78739999999993,
 -15.509600000000034,
 375.9920000000002,
 27.526400000000006,
 28.554599999999994,
 -6.3152000000000115,
 599.7414000000006,
 -11940.210400000002,
 53.47,
 764.5541999999999,
 52.13259999999997,
 -305.8925999999999,
 -46.23760000000002,
 343.57520000000017,
 20.622599999999995,
 -21.250199999999722,
 -64.03980000000004,
 113.74900000000008,
 -879.9873999999999,
 -11.372999999999877,
 -233.97639999999978,
 274.95480000000066,
 -33.99180000000018,
 259.24580000000003,
 -1815.7556,
 -2083.495600000001,
 112.77739999999989,
 1731.0722000000096,
 -22.228999999999992,
 -9.710399999999993,
 3273.6853999999985,
 -284.9482000000001,
 29477.763400000003,
 -406.2814000000002,
 -313.6537999999998,
 -10.954000000000006,
 -46.90360000000001,
 -43.333399999999955,
 138.98260000000008,
 -3171.098199999999,
 47.89120000000008,
 275.9079999999997,
 -409.6904000000003,
 678.5346000000002,
 7.823599999999942,
 393.05960000000033,
 83.31500000000004,
 146.85300000000018,
 297.8551999

In [57]:
p_value_dict_paired

{'HIST3H2A': 4.0436066512253253e-08,
 'LIN7B': 0.2891645967596114,
 'LXN': 0.0002322367302316609,
 'CNKSR2': 3.420577429814986e-12,
 'SCML1': 0.06251345969931547,
 'AC024592.12': 0.08658823731308368,
 'GSDMD': 3.041721155197623e-06,
 'AKR1C1': 1.9385746866608284e-05,
 'C3orf62': 4.768559345720827e-11,
 'CRISPLD2': 1.3766122667116988e-05,
 'DOCK5': 0.06415312266683633,
 'SLC33A1': 7.584453673521912e-10,
 'GLI1': 0.02130179835131911,
 'STK17B': 4.51183332012357e-13,
 'RP5-966M1.6': 1.6678484321488395e-09,
 'VPS52': 0.6148682201927615,
 'HAGHL': 0.004337619102628813,
 'CDC37L1': 6.001366612356978e-12,
 'KRT10': 0.0005784370532957748,
 'AEN': 0.652979252379783,
 'CYHR1': 0.00013726998677642208,
 'VWA1': 0.3726066525613332,
 'EXOSC9': 0.18811884631854897,
 'TNFAIP8L2': 1.795963782588147e-18,
 'PFDN2': 2.7764211853232284e-13,
 'TCP1': 6.667076623632467e-10,
 'ALG9': 9.061554520749966e-08,
 'RPS15A': 0.6204247342604798,
 'PMEL': 0.01460936770378406,
 'SFMBT1': 0.26141464389270735,
 'CYBA': 8.

In [58]:
len(p_value_dict_paired)

17653

In [59]:
rejected_genes_paired = {}
rejected_gene_names_paired=[]
for gene, p_value in p_value_dict_paired.items():
    if (p_value < 0.05):
        rejected_genes_paired[gene] = 'Reject H0'
        rejected_gene_names_paired.append(gene)

In [60]:
rejected_genes_paired

{'HIST3H2A': 'Reject H0',
 'LXN': 'Reject H0',
 'CNKSR2': 'Reject H0',
 'GSDMD': 'Reject H0',
 'AKR1C1': 'Reject H0',
 'C3orf62': 'Reject H0',
 'CRISPLD2': 'Reject H0',
 'SLC33A1': 'Reject H0',
 'GLI1': 'Reject H0',
 'STK17B': 'Reject H0',
 'RP5-966M1.6': 'Reject H0',
 'HAGHL': 'Reject H0',
 'CDC37L1': 'Reject H0',
 'KRT10': 'Reject H0',
 'CYHR1': 'Reject H0',
 'TNFAIP8L2': 'Reject H0',
 'PFDN2': 'Reject H0',
 'TCP1': 'Reject H0',
 'ALG9': 'Reject H0',
 'PMEL': 'Reject H0',
 'CYBA': 'Reject H0',
 'GINS1': 'Reject H0',
 'GPX3': 'Reject H0',
 'IPO9': 'Reject H0',
 'SHKBP1': 'Reject H0',
 'ARHGAP40': 'Reject H0',
 'ENO3': 'Reject H0',
 'COX19': 'Reject H0',
 'LMF1': 'Reject H0',
 'RDH14': 'Reject H0',
 'GDE1': 'Reject H0',
 'LSM14B': 'Reject H0',
 'TLE2': 'Reject H0',
 'EIF4EBP3': 'Reject H0',
 'LRP2BP': 'Reject H0',
 'PELO': 'Reject H0',
 'CDIPT': 'Reject H0',
 'C19orf44': 'Reject H0',
 'PABPN1': 'Reject H0',
 'ZWINT': 'Reject H0',
 'DYNLL1': 'Reject H0',
 'NRXN3': 'Reject H0',
 'FYTTD1'

In [61]:
len(rejected_genes_paired)

12892

In [62]:
rejected_gene_names_paired

['HIST3H2A',
 'LXN',
 'CNKSR2',
 'GSDMD',
 'AKR1C1',
 'C3orf62',
 'CRISPLD2',
 'SLC33A1',
 'GLI1',
 'STK17B',
 'RP5-966M1.6',
 'HAGHL',
 'CDC37L1',
 'KRT10',
 'CYHR1',
 'TNFAIP8L2',
 'PFDN2',
 'TCP1',
 'ALG9',
 'PMEL',
 'CYBA',
 'GINS1',
 'GPX3',
 'IPO9',
 'SHKBP1',
 'ARHGAP40',
 'ENO3',
 'COX19',
 'LMF1',
 'RDH14',
 'GDE1',
 'LSM14B',
 'TLE2',
 'EIF4EBP3',
 'LRP2BP',
 'PELO',
 'CDIPT',
 'C19orf44',
 'PABPN1',
 'ZWINT',
 'DYNLL1',
 'NRXN3',
 'FYTTD1',
 'CAPRIN1',
 'AOC3',
 'KIFC3',
 'DOCK11',
 'TMEM231',
 'DNAH17',
 'DCHS1',
 'ZFYVE19',
 'HS3ST4',
 'SLC5A8',
 'WDR53',
 'NLN',
 'TBC1D31',
 'ZNF597',
 'RNF114',
 'SFRP2',
 'PNMA2',
 'C1orf158',
 'HENMT1',
 'STX16',
 'BFAR',
 'SHROOM3',
 'NUP107',
 'HS6ST1',
 'BRWD3',
 'FGF11',
 'CCDC138',
 'FAM193B',
 'BATF',
 'AGFG2',
 'TVP23B',
 'BCL6',
 'DNAJC3',
 'C1orf233',
 'ADH6',
 'ITGB6',
 'CEACAM21',
 'ETV6',
 'C1RL',
 'CCDC181',
 'GPN3',
 'RAP1GAP',
 'MAGED2',
 'C1orf85',
 'JRKL',
 'BHLHE40',
 'NMUR1',
 'SPATA18',
 'SLC17A3',
 'NTPCR',
 'CBWD3'

In [64]:
#FDR Correction Method

In [65]:
P = []
gene_names = []

In [66]:
for gene, p_value in p_value_dict_paired.items():
    P.append(p_value)
    gene_names.append(gene)

In [67]:
P

[4.0436066512253253e-08,
 0.2891645967596114,
 0.0002322367302316609,
 3.420577429814986e-12,
 0.06251345969931547,
 0.08658823731308368,
 3.041721155197623e-06,
 1.9385746866608284e-05,
 4.768559345720827e-11,
 1.3766122667116988e-05,
 0.06415312266683633,
 7.584453673521912e-10,
 0.02130179835131911,
 4.51183332012357e-13,
 1.6678484321488395e-09,
 0.6148682201927615,
 0.004337619102628813,
 6.001366612356978e-12,
 0.0005784370532957748,
 0.652979252379783,
 0.00013726998677642208,
 0.3726066525613332,
 0.18811884631854897,
 1.795963782588147e-18,
 2.7764211853232284e-13,
 6.667076623632467e-10,
 9.061554520749966e-08,
 0.6204247342604798,
 0.01460936770378406,
 0.26141464389270735,
 8.117188180763698e-08,
 1.8797654237813683e-15,
 6.137067474564373e-20,
 9.158591946660018e-07,
 0.03756992993083354,
 0.0286271291816173,
 0.01658550843819432,
 5.6978566367483e-05,
 1.7526988675838441e-12,
 0.20218142264702482,
 0.030776565745962332,
 6.9973912702117515e-06,
 1.3888458948117415e-08,
 7

In [68]:
gene_names

['HIST3H2A',
 'LIN7B',
 'LXN',
 'CNKSR2',
 'SCML1',
 'AC024592.12',
 'GSDMD',
 'AKR1C1',
 'C3orf62',
 'CRISPLD2',
 'DOCK5',
 'SLC33A1',
 'GLI1',
 'STK17B',
 'RP5-966M1.6',
 'VPS52',
 'HAGHL',
 'CDC37L1',
 'KRT10',
 'AEN',
 'CYHR1',
 'VWA1',
 'EXOSC9',
 'TNFAIP8L2',
 'PFDN2',
 'TCP1',
 'ALG9',
 'RPS15A',
 'PMEL',
 'SFMBT1',
 'CYBA',
 'GINS1',
 'GPX3',
 'IPO9',
 'SHKBP1',
 'ARHGAP40',
 'ENO3',
 'COX19',
 'LMF1',
 'S100P',
 'RDH14',
 'GDE1',
 'LSM14B',
 'TLE2',
 'ACOT8',
 'EIF4EBP3',
 'LRP2BP',
 'PELO',
 'CDIPT',
 'C19orf44',
 'PABPN1',
 'NFKBIB',
 'ZWINT',
 'DYNLL1',
 'NRXN3',
 'FYTTD1',
 'CAPRIN1',
 'AOC3',
 'KIFC3',
 'DOCK11',
 'NENF',
 'TMEM231',
 'DNAH17',
 'DCHS1',
 'ZFYVE19',
 'NTRK1',
 'CCDC62',
 'HS3ST4',
 'SLC5A8',
 'WDR53',
 'NLN',
 'TBC1D31',
 'NOXRED1',
 'ZNF597',
 'RNF114',
 'JOSD2',
 'SFRP2',
 'PNMA2',
 'CDH10',
 'C1orf158',
 'HENMT1',
 'STX16',
 'PTX3',
 'BFAR',
 'MEP1B',
 'SHROOM3',
 'NUP107',
 'HS6ST1',
 'BRWD3',
 'FGF11',
 'CCDC138',
 'FAM193B',
 'BATF',
 'COL4A2',
 'AG

In [69]:
len(P)

17653

In [70]:
len(gene_names)

17653

In [71]:
q_fdr = multi.multipletests(P, method = 'fdr_bh')[1]

In [72]:
q_fdr

array([1.47544002e-07, 3.37139068e-01, 4.63606808e-04, ...,
       2.20673833e-12, 2.46860796e-10, 3.38139689e-06])

In [73]:
len(q_fdr)

17653

In [74]:
gene_names_modified = [gene_names[x] for x in range(len(P)) if q_fdr[x]<=0.05]

In [75]:
gene_names_modified

['HIST3H2A',
 'LXN',
 'CNKSR2',
 'GSDMD',
 'AKR1C1',
 'C3orf62',
 'CRISPLD2',
 'SLC33A1',
 'GLI1',
 'STK17B',
 'RP5-966M1.6',
 'HAGHL',
 'CDC37L1',
 'KRT10',
 'CYHR1',
 'TNFAIP8L2',
 'PFDN2',
 'TCP1',
 'ALG9',
 'PMEL',
 'CYBA',
 'GINS1',
 'GPX3',
 'IPO9',
 'ARHGAP40',
 'ENO3',
 'COX19',
 'LMF1',
 'RDH14',
 'GDE1',
 'LSM14B',
 'TLE2',
 'EIF4EBP3',
 'LRP2BP',
 'PELO',
 'CDIPT',
 'C19orf44',
 'PABPN1',
 'ZWINT',
 'DYNLL1',
 'NRXN3',
 'FYTTD1',
 'CAPRIN1',
 'AOC3',
 'KIFC3',
 'DOCK11',
 'TMEM231',
 'DNAH17',
 'DCHS1',
 'ZFYVE19',
 'HS3ST4',
 'SLC5A8',
 'WDR53',
 'NLN',
 'TBC1D31',
 'ZNF597',
 'RNF114',
 'SFRP2',
 'PNMA2',
 'C1orf158',
 'HENMT1',
 'STX16',
 'BFAR',
 'SHROOM3',
 'NUP107',
 'HS6ST1',
 'BRWD3',
 'FGF11',
 'CCDC138',
 'FAM193B',
 'BATF',
 'AGFG2',
 'TVP23B',
 'BCL6',
 'DNAJC3',
 'C1orf233',
 'ADH6',
 'ITGB6',
 'CEACAM21',
 'ETV6',
 'C1RL',
 'CCDC181',
 'GPN3',
 'RAP1GAP',
 'MAGED2',
 'C1orf85',
 'JRKL',
 'BHLHE40',
 'NMUR1',
 'SPATA18',
 'SLC17A3',
 'NTPCR',
 'CBWD3',
 'TLDC2',

In [76]:
#number of rejected genes after correction
len(gene_names_modified)

12570

In [77]:
#number of rejected genes before correction
len(rejected_genes_paired)

12892

In [85]:
intersection_paired=[]
intersection_paired=np.intersect1d(rejected_gene_names_paired,gene_names_modified)

In [86]:
len(intersection_paired)

12570

In [87]:
intersection_paired

array(['A2M', 'A2ML1', 'A4GALT', ..., 'ZZEF1', 'ZZZ3', 'hsa-mir-1199'],
      dtype='<U17')

In [89]:
difference_paired=[]
difference_paired = np.setdiff1d(rejected_gene_names_paired,gene_names_modified)

In [90]:
len(difference_paired)

322